In [1]:
import pandas as pd

In [2]:
import psycopg2

In [3]:
import sqlalchemy as db
engine = db.create_engine('postgresql://suhan:suhan005@db:5432/freedom')

In [4]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String
meta = MetaData()

students = Table(
   'students', meta, 
   Column('id', Integer, primary_key = True), 
   Column('name', String), 
   Column('lastname', String),
)
meta.create_all(engine)

In [5]:
ins = students.insert()
ins = students.insert().values(name = 'Ravi', lastname = 'Kapoor')
conn = engine.connect()
result = conn.execute(ins)

In [6]:
s = students.select()
conn = engine.connect()
result = conn.execute(s)

for row in result:
    print (row)

(1, 'Ravi', 'Kapoor')
(2, 'Ravi', 'Kapoor')


In [9]:
from werkzeug.security import generate_password_hash

In [10]:
from flask_login import LoginManager

In [11]:
from flask_login import UserMixin

In [14]:
from flask import Flask

In [1]:
from sqlalchemy import Table, Column, Integer, ForeignKey
from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base

In [1]:
from flask_login import UserMixin
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base
from flask import Flask

Base = declarative_base()

db = SQLAlchemy()

class User(UserMixin, db.Model, Base):
    __tablename__ = 'users'
    id = db.Column(db.Integer, primary_key=True) # primary keys are required by SQLAlchemy
    email = db.Column(db.String(100), unique=True)
    password = db.Column(db.String(512))
    name = db.Column(db.String(100))
    mobile = db.Column(db.String(25), unique=True)
    broker_id = db.Column(db.String(25), unique=True)
    broker_name = db.Column(db.String(50))
    api_key = db.Column(db.String(512), unique=True)
    api_secret = db.Column(db.String(512), unique=True)
    api_token = db.Column(db.String(512), unique=True)
    session_id = db.Column(db.String(512), unique=True)
    algos = relationship("Algos", back_populates="users")
    portfolios = relationship("Portfolios", back_populates="users")
    jobs = relationship("Jobs", back_populates="users")


class Algos(db.Model, Base):
    __tablename__ = 'algos'
    id = db.Column(db.Integer, primary_key=True) # primary keys are required by SQLAlchemy
    title = db.Column(db.String(100))
    algo = db.Column(db.String(100))
    user_id = db.Column(db.Integer, db.ForeignKey('users.id'), index=True)
    users = relationship("User", back_populates="algos")


class Portfolios(db.Model, Base):
    __tablename__ = 'portfolios'
    id = db.Column(db.Integer, primary_key=True) # primary keys are required by SQLAlchemy
    title = db.Column(db.String(100), index=True)
    stock = db.Column(db.String(20))
    user_id = db.Column(db.Integer, db.ForeignKey('users.id'), index=True)
    users = relationship("User", back_populates="portfolios")


class Jobs(db.Model, Base):
    __tablename__ = 'jobs'
    id = db.Column(db.Integer, primary_key=True) # primary keys are required by SQLAlchemy
    job_id = db.Column(db.String(100), unique=True)
    job_type = db.Column(db.String(25))
    job_status = db.Column(db.String(25))
    user_id = db.Column(db.Integer, db.ForeignKey('users.id'), index=True)
    users = relationship("User", back_populates="jobs")
    trades = relationship("Trades", back_populates="jobs")


class Trades(db.Model, Base):
    __tablename__ = 'trades'
    id = db.Column(db.Integer, primary_key=True) # primary keys are required by SQLAlchemy
    timestamp  = db.Column(db.Integer)
    stock = db.Column(db.String(20), index=True)
    price = db.Column(db.Float)
    qty = db.Column(db.Float)
    buy_or_sell = db.Column(db.String(20))
    en_or_ex = db.Column(db.String(20))
    order_id = db.Column(db.String(100))
    job_id = db.Column(db.Integer, db.ForeignKey('jobs.id'), index=True)
    jobs = relationship("Jobs", back_populates="trades")


class OHLC(db.Model, Base):
    __tablename__ = 'ohlc'
    id = db.Column(db.Integer, primary_key=True) # primary keys are required by SQLAlchemy
    stock = db.Column(db.String(20), index=True)
    timestamp  = db.Column(db.Integer)
    open = db.Column(db.Float)
    high = db.Column(db.Float)
    low = db.Column(db.Float)
    close = db.Column(db.Float)
    volume = db.Column(db.Float)

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'postgresql://suhan:suhan005@db:5432/freedom'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db.init_app(app)


##########
from sqlalchemy import Column, Integer, String, create_engine, select
from sqlalchemy.orm import sessionmaker
import pandas as pd

engine = create_engine('postgresql://suhan:suhan005@db:5432/freedom', echo=False)
#Base = declarative_base(bind=engine)
Session = sessionmaker(bind=engine)
session = Session()

conn = session.bind

In [4]:
df_query = select([User])

df_data = pd.read_sql(df_query, con = conn)
df_data

,id,email,password,name,mobile,broker_id,broker_name,api_key,api_secret,api_token,session_id
0,1,suhansaha@gmail.com,sha256$ZPzKNGp4$487b1d21cd69cebcd9dd7022bbba4d...,Suhan Saha,+917984183480,YM1755,ZERODHA,b2w0sfnr1zr92nxm,jtga2mp2e5fn29h8w0pe2kb722g3dh1q,None,None


In [3]:
with app.app_context():
    db.create_all()
    user = User.query.filter_by(email='suhansaha@gmail.com').first()
    user.name = 'Suhan Saha'
    user.mobile = '+917984183480'
    user.broker_name = 'ZERODHA'
    user.broker_id = 'YM1755'
    user.api_key = 'b2w0sfnr1zr92nxm'
    user.api_secret = 'jtga2mp2e5fn29h8w0pe2kb722g3dh1q'
    db.session.commit()